In [ ]:
import joblib
import pandas as pd
# Загрузка модели из файла
automl_il2 = joblib.load("/Users/rleontiev/Downloads/Telegram Desktop/il2_model_pos_features.pkl")
automl_il4 = joblib.load("/Users/rleontiev/Downloads/Telegram Desktop/il_4_LightAutoML_feature_selection_corr_importance_accurate (2).pkl")
automl_il10 = joblib.load("/Users/rleontiev/Downloads/Telegram Desktop/model_all_features_acc_importance_63.pkl")

features_il2 = pd.read_csv('/Users/rleontiev/Downloads/Telegram Desktop/il2_pos_features.csv')['0'].to_list() + ['Epitope ID']
features_il4 = pd.read_csv('/Users/rleontiev/Downloads/Telegram Desktop/IL-4_feature_importance_accurate_filtered.csv')['Feature'].to_list() + ['Epitope ID']
features_il10 = pd.read_csv('/Users/rleontiev/Downloads/features_il10.csv')['feature'].to_list() + ['Epitope ID']
# features_il10 = pd.read_csv('')

In [51]:
train_df = pd.read_csv('/Users/rleontiev/Downloads/Telegram Desktop/overall_train.csv').sort_values(by='Epitope ID')
test = pd.read_csv('/Users/rleontiev/Downloads/Telegram Desktop/overall_test.csv').sort_values(by='Epitope ID')

In [52]:
train_df = pd.concat([train_df, test], axis=0)

In [56]:
sum(train_df['IFNg release'].isna())

1877

In [57]:
# train_df = pd.read_csv('/Users/rleontiev/Downloads/Telegram Desktop/overall_train.csv').sort_values(by='Epitope ID')

ifng_preds = pd.read_csv('/Users/rleontiev/Downloads/Telegram Desktop/IFNg_release_na_epitopes_prediction.csv')
ifng_preds['IFNg release'] = ifng_preds["Prediction"].replace({"IFN-γ inducer": 1, "Non-inducer": 0})
ifng_preds.rename({'Seq_ID': 'Epitope ID'}, axis=1, inplace=True)
train_df = train_df.merge(
    ifng_preds[['Epitope ID', 'IFNg release']],  # Оставляем только нужные колонки
    on='Epitope ID',
    how='left',
    suffixes=('', '_new')  # Новый столбец 'IFNg release_new'
)

# Заполняем пропущенные значения в основном датафрейме, если есть данные из df_small
train_df['IFNg release'] = train_df['IFNg release'].combine_first(train_df['IFNg release_new'])

# Удаляем временный столбец
train_df = train_df.drop(columns=['IFNg release_new'])

# Удаляем строки, где 'IFNg release' так и остался NaN
train_df = train_df.dropna(subset=['IFNg release'])

train_df_y = train_df[['IL-2 release', 'IL-4 release', 'IL-10 release', 'IFNg release']]

/var/folders/3m/ml853yhj7m1gwt_yly8mfyd40000gp/T/ipykernel_81489/2545224998.py:4: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [58]:
train_df

,Epitope ID,MHC Allele,Evidence Code,MHC Class,Epitope Seq,Host,Antigen Source,IL-2 release,IL-4 release,IL-10 release,IFNg release
0,29,HLA class II,T cell assay -T cell subset identification,II,AAAEGVGKDNKLSVLLFTTQ,Homo sapiens (human),Vaccinia virus,NaN,NaN,NaN,0.0
1,39,HLA-DRB1*04:03,MHC binding prediction,II,AAAIFMTATPPGTAD,Homo sapiens (human),Dengue virus,NaN,NaN,NaN,1.0
2,46,HLA class II,MHC binding prediction,II,AAALEQLLGQTADVA,Homo sapiens (human),Mycobacterium leprae,NaN,NaN,NaN,1.0
3,60,HLA class II,T cell assay -Biological process measured,II,AAASAIQGNVTSIHSL,Homo sapiens (human),Mycobacterium tuberculosis,NaN,1.0,NaN,1.0
4,61,HLA class II,T cell assay -T cell subset identification,II,AAASKVKANLLSVEE,Homo sapiens (human),Hepacivirus hominis,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...
41290,2268867,HLA-DRB1*15:01,MHC binding prediction,II,RHPLFQPLHTVMRET,Homo sapiens (human),human gammaherpesvirus 4,NaN,NaN,NaN,0.0
41291,2268926,HLA-DRB1*15:01,MHC binding prediction,II,VLAIILYFIAFALGIF,Homo sapiens (human),human gammaherpesvirus 4,NaN,NaN,NaN,0.0
41292,2268989,HLA-DR4,Cited reference,II,AQNILLSNAPLGPQFP,Homo sapiens (human),NaN,0.0,0.0,0.0,0.0
41293,2269660,HLA class II,T cell assay -T cell subset identification,II,LLFLLFSLGWVGPSR,Homo sapiens (human),Homo sapiens,NaN,NaN,NaN,1.0


In [59]:
all_df = pd.read_csv('/Users/rleontiev/Downloads/Telegram Desktop/features_human_cytokine.csv').sort_values(by='Epitope ID')

In [169]:
test_il2 = pd.read_csv('/Users/rleontiev/Downloads/Telegram Desktop/il2_test.csv').sort_values(by='Epitope ID')
test_il2_y = test_il2['IL-2 release']
test_il4 = pd.read_csv('/Users/rleontiev/Downloads/Telegram Desktop/il4_test.csv').sort_values(by='Epitope ID')
test_il4_y = test_il4['IL-4 release']
test_il10 = pd.read_csv('/Users/rleontiev/Downloads/Telegram Desktop/il10_test.csv').sort_values(by='Epitope ID')
test_il10_y = test_il10['IL-10 release']

test_il2 = all_df[all_df['Epitope ID'].isin(test_il2['Epitope ID'])].drop(['Epitope ID'], axis=1)
test_il2['IL-2 release'] = test_il2_y.to_list()
test_il4 = all_df[all_df['Epitope ID'].isin(test_il4['Epitope ID'])].drop(['Epitope ID'], axis=1)
test_il4['IL-4 release'] = test_il4_y.to_list()
test_il10 = all_df[all_df['Epitope ID'].isin(test_il10['Epitope ID'])].drop(['Epitope ID'], axis=1)
test_il10['IL-10 release'] = test_il10_y.to_list()

In [171]:
test_il2.to_csv('/Users/rleontiev/Downloads/il2_test_features.csv')
test_il4.to_csv('/Users/rleontiev/Downloads/il4_test_features.csv')
test_il10.to_csv('/Users/rleontiev/Downloads/il10_test_features.csv')

In [60]:
train_df = all_df[all_df['Epitope ID'].isin(train_df['Epitope ID'])]

In [93]:
train_df_y

,IL-2 release,IL-4 release,IL-10 release,IFNg release
0,NaN,NaN,NaN,0.0
1,NaN,NaN,NaN,1.0
2,NaN,NaN,NaN,1.0
3,NaN,1.0,NaN,1.0
4,NaN,NaN,NaN,0.0
...,...,...,...,...
32723,NaN,NaN,NaN,1.0
32724,NaN,NaN,NaN,1.0
32725,NaN,NaN,NaN,1.0
32726,NaN,NaN,NaN,1.0


In [94]:
train_df_y['IFNg release'].info()

<class 'pandas.core.series.Series'>
Index: 31713 entries, 0 to 32727
Series name: IFNg release
Non-Null Count  Dtype  
--------------  -----  
31713 non-null  float64
dtypes: float64(1)
memory usage: 495.5 KB


In [61]:
train_df['IFNg release'] = train_df_y['IFNg release'].to_list()
train_df['IL-10 release'] = train_df_y['IL-10 release'].to_list()
# train_df['IL-4 release'] = train_df_y['IL-4 release'].to_list()
# train_df['IL-2 release'] = train_df_y['IL-2 release'].to_list()

/var/folders/3m/ml853yhj7m1gwt_yly8mfyd40000gp/T/ipykernel_81489/3638473451.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/3m/ml853yhj7m1gwt_yly8mfyd40000gp/T/ipykernel_81489/3638473451.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [139]:
il2_to_pred = train_df[train_df['IL-2 release'].isna()][features_il2]
il4_to_pred = train_df[train_df['IL-4 release'].isna()][features_il4]
il10_to_pred = train_df[train_df['IL-10 release'].isna()][features_il10]

In [151]:
il2_to_pred['IL-2 release'] = list(automl_il2.predict(il2_to_pred.drop(['Epitope ID'], axis=1)).data.flatten())
il4_to_pred['IL-4 release'] = list(automl_il4.predict(il4_to_pred.drop(['Epitope ID'], axis=1)).data.flatten())
il10_to_pred['IL-10 release'] = list(automl_il10.predict(il10_to_pred.drop(['Epitope ID'], axis=1)).data.flatten())



# il2_to_pred['predict'] = list(automl_il2.predict(il2_to_pred.drop(['Epitope ID'], axis=1)).data.flatten())
# il4_to_pred['predict'] = list(automl_il4.predict(il4_to_pred.drop(['Epitope ID'], axis=1)).data.flatten())
# il10_to_pred['predict'] = list(automl_il10.predict(il10_to_pred.drop(['Epitope ID'], axis=1)).data.flatten())

In [158]:
# il10_to_pred
pseudo_label_df_il10_confident = il10_to_pred[(il10_to_pred['IL-10 release'] >= 0.87) | (il10_to_pred['IL-10 release'] <= 0.06)]

In [159]:
pseudo_label_df_il10_confident['IL-10 release'] = (pseudo_label_df_il10_confident['IL-10 release'] > 0.5).astype(int)
pseudo_label_df_il10_confident.drop(['Epitope ID'], axis=1).to_csv('pseudo_label_il_10.csv')

/var/folders/3m/ml853yhj7m1gwt_yly8mfyd40000gp/T/ipykernel_79075/3175466150.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pseudo_label_df_il10_confident['IL-10 release'] = (pseudo_label_df_il10_confident['IL-10 release'] > 0.5).astype(int)


In [160]:
pseudo_label_df_il10_confident['IL-10 release'].value_counts()

IL-10 release
0    633
1    607
Name: count, dtype: int64

In [141]:

il2_to_pred['IL-2 release'] = (il2_to_pred['IL-2 release'] > 0.5).astype(int)
il4_to_pred['IL-4 release'] = (il4_to_pred['IL-4 release'] > 0.5).astype(int)
il10_to_pred['IL-10 release'] = (il10_to_pred['IL-10 release'] > 0.5).astype(int)

In [142]:
train_df['IFNg release'].value_counts()

IFNg release
0.0    22834
1.0     8879
Name: count, dtype: int64

In [100]:
il2_to_pred['IL-2 release'].value_counts()

IL-2 release
1    30823
0      628
Name: count, dtype: int64

In [101]:
def merg(train_df, ifng_preds, target):
    train_df = train_df.merge(
        ifng_preds[['Epitope ID', target]],  # Оставляем только нужные колонки
        on='Epitope ID',
        how='left',
        suffixes=('', '_new')  # Новый столбец 'IFNg release_new'
    )

    # Заполняем пропущенные значения в основном датафрейме, если есть данные из df_small
    train_df[target] = train_df[target].combine_first(train_df[target + '_new'])

    # Удаляем временный столбец
    train_df = train_df.drop(columns=[target + '_new'])
    return train_df

In [143]:
train_df = merg(train_df, il2_to_pred, 'IL-2 release')

In [144]:
train_df = merg(train_df, il4_to_pred, 'IL-4 release')
train_df = merg(train_df, il10_to_pred, 'IL-10 release')

In [161]:
train_df.to_csv('/Users/rleontiev/Downloads/labeled_overall_train.csv')

In [163]:
train_df

,Epitope ID,Moreau-Broto_Auto_MBAuto_CIDH920105_1,Moreau-Broto_Auto_MBAuto_CIDH920105_2,Moreau-Broto_Auto_MBAuto_CIDH920105_3,Moreau-Broto_Auto_MBAuto_CIDH920105_4,Moreau-Broto_Auto_MBAuto_CIDH920105_5,Moreau-Broto_Auto_MBAuto_CIDH920105_6,Moreau-Broto_Auto_MBAuto_CIDH920105_7,Moreau-Broto_Auto_MBAuto_CIDH920105_8,Moreau-Broto_Auto_MBAuto_CIDH920105_9,...,molar_extinction_reduced,flexibility,instability_index,Hydrophobic Moment,Boman,MS-WHIM,IFNg release,IL-10 release,IL-4 release,IL-2 release
0,29,0.188983,0.335746,0.450658,0.649392,0.793122,0.885213,0.914524,1.041162,1.292689,...,0,1.014657,4.835000,0.325786,0.711000,1031.562561,0.0,1.0,0.0,1.0
1,39,0.281813,0.596036,0.953489,1.297521,1.688596,2.160669,2.724423,3.433578,4.360832,...,0,0.990044,8.933333,0.157444,-0.254000,717.850291,1.0,0.0,0.0,1.0
2,46,0.020947,0.054307,0.191502,0.340707,0.508782,0.587770,0.539570,0.613145,0.542841,...,0,0.995159,16.513333,0.412295,0.026000,735.893541,1.0,0.0,1.0,1.0
3,60,-0.245354,-0.445723,-0.666538,-0.941086,-1.095475,-1.382599,-1.632492,-1.831190,-2.209876,...,0,1.000527,6.475000,0.320969,0.164375,770.409896,1.0,1.0,1.0,1.0
4,61,-0.136385,-0.420870,-0.646164,-0.811573,-0.898015,-1.115934,-1.273519,-1.491801,-1.793890,...,0,1.006250,23.486667,0.276554,0.866667,765.430291,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31708,2270776,0.293405,0.576362,1.086273,1.858974,2.647398,3.533246,4.601058,6.100668,7.843729,...,6990,0.965196,124.873333,0.278074,2.506000,938.531211,1.0,0.0,1.0,1.0
31709,2270777,0.291379,0.571029,0.911281,1.262918,1.578233,1.888806,2.306298,2.932467,3.533812,...,2980,0.982638,15.283333,0.509844,0.678333,1118.068655,1.0,1.0,1.0,1.0
31710,2270779,-0.194652,-0.397718,-0.615231,-0.824286,-1.091001,-1.397698,-1.739588,-2.037785,-2.319942,...,0,0.989971,96.695238,0.612526,0.306667,1244.069699,1.0,1.0,1.0,1.0
31711,2270786,-0.222909,-0.416274,-0.574186,-0.662357,-0.857433,-1.006743,-1.070539,-1.217096,-1.465845,...,5500,1.002589,9.400000,0.480024,0.734444,965.541436,1.0,1.0,1.0,1.0


In [164]:
df = pd.read_csv('/Users/rleontiev/Downloads/Telegram Desktop/overall_train.csv').sort_values(by='Epitope ID')

In [ ]:
df['IFNg release'] = train_df['IFNg release'].to_list()
df['IL-10 release'] = train_df['IL-10 release'].to_list()
df['IL-4 release'] = train_df['IL-4 release'].to_list()
df['IL-2 release'] = train_df['IL-2 release'].to_list()

In [19]:
import pandas as pd
df_train = pd.read_csv("/Users/rleontiev/Downloads/labeled_overall_train.csv")

In [73]:
train_df_il10 = pd.read_csv('/Users/rleontiev/Downloads/Telegram Desktop/il10_train.csv').sort_values(by='Epitope ID')
train_y = train_df_il10['IL-10 release'].to_list()
test_df_il10 = pd.read_csv('/Users/rleontiev/Downloads/Telegram Desktop/il10_test.csv').sort_values(by='Epitope ID')
test_y = test_df_il10['IL-10 release'].to_list()
train_df_il10 = train_df[train_df['Epitope ID'].isin(train_df_il10['Epitope ID'])]
test_df_il10 = train_df[train_df['Epitope ID'].isin(test_df_il10['Epitope ID'])]

In [75]:
train_df_il10['IL-10 release'] = train_y[:len(train_df_il10)]
test_df_il10['IL-10 release'] = test_y[:len(test_df_il10)]

/var/folders/3m/ml853yhj7m1gwt_yly8mfyd40000gp/T/ipykernel_81489/2765955996.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/3m/ml853yhj7m1gwt_yly8mfyd40000gp/T/ipykernel_81489/2765955996.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [68]:
train_df_il10

,Epitope ID,Moreau-Broto_Auto_MBAuto_CIDH920105_1,Moreau-Broto_Auto_MBAuto_CIDH920105_2,Moreau-Broto_Auto_MBAuto_CIDH920105_3,Moreau-Broto_Auto_MBAuto_CIDH920105_4,Moreau-Broto_Auto_MBAuto_CIDH920105_5,Moreau-Broto_Auto_MBAuto_CIDH920105_6,Moreau-Broto_Auto_MBAuto_CIDH920105_7,Moreau-Broto_Auto_MBAuto_CIDH920105_8,Moreau-Broto_Auto_MBAuto_CIDH920105_9,...,QSO_99,shannon_entropy,molar_extinction_reduced,flexibility,instability_index,Hydrophobic Moment,Boman,MS-WHIM,IFNg release,IL-10 release
3865,888,0.052586,0.107602,0.165454,0.271932,0.437479,0.588578,0.849816,1.287230,1.667607,...,0.0,3.239098,0,1.021741,59.188889,0.440078,3.586111,1014.545236,0.0,1.0
17234,8399,0.073013,0.118453,0.180685,0.243500,0.372710,0.474580,0.474594,0.741879,1.252689,...,0.0,3.189898,1490,1.013089,38.066667,0.446569,1.969333,830.954471,0.0,0.0
3031,8930,0.461405,0.904821,1.361863,1.837785,2.296789,2.745736,3.418948,4.375279,5.450154,...,0.0,2.858459,0,1.022022,6.162500,0.293016,4.606875,960.013251,0.0,1.0
1841,10269,0.277079,0.659350,1.034084,1.397027,1.778584,2.167702,2.626678,3.192112,3.998377,...,0.0,3.625000,0,1.003694,15.225000,0.406050,0.820625,836.458531,1.0,1.0
17235,16348,0.336238,0.766466,1.257792,1.863942,2.469072,3.310733,4.227452,5.246295,6.418325,...,0.0,3.189898,0,1.000806,19.333333,0.463154,0.312667,862.474496,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22948,2233349,-0.257475,-0.609654,-0.947556,-1.224186,-1.513605,-1.868819,-2.278560,-2.763566,-3.424730,...,0.0,3.572431,2980,0.998353,11.883333,0.448731,0.909444,1061.570196,0.0,1.0
23002,2238281,-0.357738,-0.642834,-1.098742,-1.532485,-2.056411,-2.712961,-3.436806,-4.268320,-5.378240,...,0.0,3.238901,0,0.985003,-21.961538,0.293419,1.520769,757.887216,0.0,1.0
41281,2269069,0.141566,0.291212,0.461266,0.655329,0.829899,1.111197,1.319390,1.710763,2.033959,...,0.0,3.522572,5500,0.980331,60.452381,0.528518,1.763333,1338.700921,1.0,1.0
41279,2269374,-0.027840,-0.140901,-0.116901,-0.004301,0.185758,0.322459,0.480251,0.681034,0.928461,...,0.0,3.189898,1490,0.961044,-5.826667,0.560156,0.312000,849.971611,1.0,1.0


In [70]:
train_df_il10.drop(['Epitope ID'], inplace=True, axis=1)

/var/folders/3m/ml853yhj7m1gwt_yly8mfyd40000gp/T/ipykernel_81489/771623543.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
train_df_il10.to_csv('/Users/rleontiev/Downloads/il10_with_ifn.csv')
test_df_il10.to_csv('/Users/rleontiev/Downloads/il10_with_ifn_test.csv')

: 